In [1]:
from dichasus_cf0x import training_set, spec
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tqdm.auto import tqdm
import scipy

2024-02-10 15:53:37.835356: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-10 15:53:38.576023: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-02-10 15:53:39.558489: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 15:53:39.582126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
training_set_positions = np.load("results/training_set_positions.npy")
test_set_positions = np.load("results/test_set_positions.npy")

In [3]:
# Use "training set" CSI to predict CSI at test set positions
training_set_csi = []

with tqdm(total = len(training_set_positions)) as pbar:
    for csi, pos, time, ds in training_set.batch(100):
        training_set_csi.append(csi.numpy())
        pbar.update(tf.shape(csi)[0].numpy())

training_set_csi = np.concatenate(training_set_csi)

  0%|          | 0/17857 [00:00<?, ?it/s]

In [4]:
# Interpolate multiple CSI datapoints, with weights s (barycentric coordinates)
# CSI must be in frequency domain!
def csi_interp(csi, s, iterations = 10):
    phi = np.zeros_like(s)
    w = None
    dims = len(csi.shape)
    #errors = []
    
    for i in range(iterations):
        w = np.einsum("n,n,n...->...", s, np.exp(-1.0j * phi), csi)
        phi = np.angle(np.einsum("a,na->n", np.conj(w.flatten()), csi.reshape(len(csi), -1)))
        #errors.append(np.sum([s[n] * np.linalg.norm(csi[n] - np.exp(1.0j * phi[n]) * w)**2 for n in range(len(csi))]))

    #plt.plot(errors)
    #plt.show()

    return w

In [5]:
def interpolate_csi_delaunay(existing_points, csi_at_existing_points, evaluation_points):
    tri = scipy.spatial.Delaunay(existing_points[:,:2])
    simplices = tri.find_simplex(evaluation_points)

    csi_interpolated = np.zeros(evaluation_points.shape[:1] + csi_at_existing_points.shape[1:], dtype = np.complex64)

    for i, simplex in enumerate(tqdm(simplices)):
        b = np.dot(tri.transform[simplex, :2], evaluation_points[i] - tri.transform[simplex, 2])
        bary_coords = np.r_[b, 1 - np.sum(b)]
        vertex_indices = tri.simplices[simplex]
        csi_vertices = csi_at_existing_points[vertex_indices]
        csi_interpolated[i] = csi_interp(csi_vertices, bary_coords)

    return csi_interpolated, (simplices >= 0)


In [6]:
test_set_csi_interpolated, test_set_csi_interpolated_valid = interpolate_csi_delaunay(training_set_positions[:,:2], training_set_csi, test_set_positions[:,:2])

  0%|          | 0/20973 [00:00<?, ?it/s]

In [7]:
np.save("results/test_set_csi_interpolated.npy", np.stack([np.real(test_set_csi_interpolated), np.imag(test_set_csi_interpolated)], axis = -1))
np.save("results/test_set_csi_interpolated_valid.npy", test_set_csi_interpolated_valid)